# AWS VPC Exercise
> I am recently starting to learn AWS. In this blog I'm gonna share how to build VPC step by step.

- toc: true
- comments: true
- branch: master
- badges: false
- categories: [AWS]
- image: images/aws-vpc.png


# Overview

The following chart shows the VPC architecture we will build in this exercise
![image.png](2021-12-16-aws_vpc/aws-vpc.png)

## Prerequisite
The resource is from the [freeCodeCamp.org](https://www.youtube.com/watch?v=g2JOHLHh4rI) presented by [digitalcloud.training]( https://digitalcloud.training). Neal did a quite good job I encourage you to watch the original video and do the exercise by yourself. Here I only logged the steps what I have done.
- Create a free [AWS Account](https://aws.amazon.com/).
- Download the [exercise code](https://digitalcloud.training/aws-vpc-deep-dive-download/)
- Install AWS Cli and config the [aws configure](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html)

## VPC Review

### VPC Components

- **Virtual Private Cloud (VPC)**: A logically isolated virtual network in the AWS cloud  
- **Subnet**: A segment of a VPC's IP address range where you can place groups of isolated resources  
- **Internet Gateway/Egress only Internet Gateway**: The Amazon VPC side of a connection to the public Internet for IPv4/IPv6  
- **Router**: Routers interconnect subnets and direct traffic between Internet gateways, virtual private gateways, NAT gateways, and subnets  
- **Peering Connection**: Direct connection between two VPCs  
- **VPC Endpoints**: Private connection to public AWS services  
- **NAT Instance**: Enables Internet access for EC2 instances in private subnets (managed by you)  
- **NAT Gateway**: Enables Internet access for EC2 instances in private subnets (managed by AWS)  
- **Virtual Private Gateway**: The Amazon VPC side of a Virtual Private Network (VPN) connection  
- **Customer Gateway**: Customer side of a VPN connection  
- **AWS Direct Connect**: High speed, high bandwidth, private network connection from customer to aws  
- **Security Group**: Instance-level firewall  
- **Network ACL**: Subnet-level firewall  

### Amazon VPC Core Knowledge
- [Difference between Internet Gateway and NAT Gateway](https://medium.com/awesome-cloud/aws-vpc-difference-between-internet-gateway-and-nat-gateway-c9177e710af6)
- A virtual private cloud (VPC) is a virtual network dedicated to your AWS account
- Analogous to having your own data center inside AWS
- It is logically isolated from other virtual networks in the AWS Cloud
- Provides complete control over the virtual networking environment including selection of IP    ranges, creation of subnets, and configuration of route tables and gateways
- You can launch your AWS resources, such as Amazon EC2 instances, into
  your VPC
- When you create a VPC, you must specify a range of IPv4
addresses for the VPC in the form of a Classless Inter-Domain
Routing (CIDR) block; for example, 10.0.0.0/16
- A VPC spans all the Availability Zones in the region
- You have full control over who has access to the AWS resources
inside your VPC
- By default you can create up to 5 VPCs per region
- A default VPC is created in each region with a subnet in each AZ

## Create VPC

Name: MyVPC  
IPv4 CIDR Block: 10.0.0.0/16  

![image.png](2021-12-16-aws_vpc/att_00000.png)

A route table was also created automatically by AWS.  
![image.png](2021-12-16-aws_vpc/att_00001.png)

## Create Subnets

Name: **Public-1A**  
Availability Zone: **us-east-1a**  
IPv4 CIDR Block: **10.0.1.0/24**  
![image.png](2021-12-16-aws_vpc/att_00003.png)

Name: **Public-1B**  
Availability Zone: **us-east-1b**  
IPv4 CIDR Block: **10.0.2.0/24**  
![image.png](2021-12-16-aws_vpc/att_00004.png)

Name: **Private-1A**  
Availability Zone: **us-east-1a**  
IPv4 CIDR Block: **10.0.3.0/24**  
![image.png](2021-12-16-aws_vpc/att_00005.png)

Name: **Private-1B**  
Availability Zone: **us-east-1b**  
IPv4 CIDR Block: **10.0.4.0/24**  
![image.png](2021-12-16-aws_vpc/att_00005.png)

Finally, they look like this.  
![image.png](2021-12-16-aws_vpc/att_00006.png)

For the public subnets, we tick **'Enable auto-assign public IPv4 address'**.  
![image.png](2021-12-16-aws_vpc/att_00007.png)

![image.png](2021-12-16-aws_vpc/att_00008.png)

## Create private route table

Name: **Private-RT**  
VPC: **MyVPC**  
Subnet associations: **Private-1A, Private-1B**  
![image.png](2021-12-16-aws_vpc/att_00009.png)

Change the name of default route table to 'MAIN' and associate the Private-1A and Private-1B to the route table.  
![image.png](2021-12-16-aws_vpc/att_00010.png)

![image.png](2021-12-16-aws_vpc/att_00011.png)

## Create Internet Gateway

Name: **MyIGW**  
VPC: **MyVPC**  

![image.png](2021-12-16-aws_vpc/att_00012.png)

![image.png](2021-12-16-aws_vpc/att_00013.png)

## Edit MAIN route table

Add **0.0.0.0/0** to internet gateway.  

![image.png](2021-12-16-aws_vpc/att_00014.png)

![image.png](2021-12-16-aws_vpc/att_00015.png)

## NAT Gateway

- Create [NAT Gateway](https://docs.aws.amazon.com/vpc/latest/userguide/vpc-nat-gateway.html)  
![image.png](2021-12-16-aws_vpc/att_00016.png)

- Go to PrivateRT and edit the route 0.0.0.0/0 to NAT Gateway  
![image.png](2021-12-16-aws_vpc/att_00017.png)

## Configure [Security Groups](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_SecurityGroups.html) and NACLs

- Create Public-Web security group  
![image.png](2021-12-16-aws_vpc/att_00018.png)

- Private-App: Private App Access, Inbound rule: **http/80 with source Public-Web, make sure the inbound comes from the web app front end in the Public-Web security group**

![image.png](2021-12-16-aws_vpc/att_00019.png)

## Launch EC2s

Use the following command to launch the EC2s. 
**aws ec2 run-instances --image-id <value> --instance-type <value> --security-group-ids <value> --subnet-id <value> --key-name <value> --user-data <value>**  

The variables that need to be filled:
- image-id: [Amazon Machine Images (AMI)](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AMIs.html), here we use ami-0ed9277fb7eb570c9
- instance-type: t2.micro
- security-group-ids: Public-Web security group.
- subnet-id: we create two EC2s in Public 1A and 1B, one EC2 in Private 1B.
- key-name: [key pairs](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html) of EC2.
- user-data are needed to be filled: the file '/user-data-subnet-id.txt' in [exercise code](https://digitalcloud.training/aws-vpc-deep-dive-download/)


    
Once we have these ready we run the following script(Make sure the aws cli installed and configured before this).  
- Launch instance in Public 1A  
aws ec2 run-instances --image-id ami-0ed9277fb7eb570c9 --instance-type t2.micro --security-group-ids sg-0ff58fb3c21c0792d --subnet-id subnet-0f0f447e902559be9 --key-name ec2_cloud --user-data file://code/user-data-subnet-id.txt
    
- Launch instance in Public 1B  
aws ec2 run-instances --image-id ami-0ed9277fb7eb570c9 --instance-type t2.micro --security-group-ids sg-0ff58fb3c21c0792d --subnet-id subnet-07266ffa901687189 --key-name ec2_cloud --user-data file://code/user-data-subnet-id.txt

- Launch instance in Private 1B  
aws ec2 run-instances --image-id ami-0ed9277fb7eb570c9 --instance-type t2.micro --security-group-ids sg-0ff58fb3c21c0792d --subnet-id subnet-0f44825a48340db38 --key-name ec2_cloud --user-data file://code/user-data-subnet-id.txt

Finally three EC2 are launched.  

![image.png](2021-12-16-aws_vpc/att_00020.png)

Visit the Instances in Public 1A and 1B.  
![image.png](2021-12-16-aws_vpc/att_00021.png)

We see the response from the server: "This instance is in the subnet wih ID: subnet-0f0f447e902559be9".  
![image.png](2021-12-16-aws_vpc/att_00022.png)

![image.png](2021-12-16-aws_vpc/att_00023.png)

Change inbound rule of Public-Web security group, set the source to "my ip". Then try it also with another IP(turn VPN on). Change it back after the experiment.  
![image.png](2021-12-16-aws_vpc/att_00024.png)

Once the VPN is turned on, the process bar was hardly moving which means the "block" was working.  
![image.png](2021-12-16-aws_vpc/att_00025.png)

Change it back after the testing.  
![image.png](2021-12-16-aws_vpc/att_00026.png)

Test the internal network with SSH.  

Ping from Public 1A to Public 1B  
![image.png](2021-12-16-aws_vpc/att_00027.png)

![image.png](2021-12-16-aws_vpc/att_00028.png)

![image.png](2021-12-16-aws_vpc/att_00029.png)

Ping from Public 1A to Private 1B  
![image.png](2021-12-16-aws_vpc/att_00030.png) 

![image.png](2021-12-16-aws_vpc/att_00031.png)

Change the security group of EC2 Private 1B to Private-App  
![image.png](2021-12-16-aws_vpc/att_00032.png) 

![image.png](2021-12-16-aws_vpc/att_00033.png)

In the Private-App, only HTTP/80 was configured that Ping(ICMP) didn't work and CURL(Http) worked from Public 1A to Private 1B  
![image.png](2021-12-16-aws_vpc/att_00034.png)

## Conclusion

I finished this exercise and it makes me understand the core concepts of VPC. Potentially I may go to get some certificates and hopefully I can plan a schedule for it.